In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import datetime

PATH = os.path.abspath('')

## Twitter tweepy client

In [ ]:
API_KEY = os.getenv('TWITTER_BOT_KEY')
API_SECRET_KEY = os.getenv('TWITTER_BOT_SECRET_KEY')

ACCESS_TOKEN = os.getenv('
ACCESS_TOKEN_SECRET = os.g

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
except:
    raise Exception("Authentication not OK")

me = api.me()
me.screen_name

## MongoDB client

In [ ]:
MONG_USER = os.getenv('MONG_USER')
MONG_PWD  = os.getenv('MONG_PWD')
MONG_HOST = os.getenv('MONG_HOST')
MONG_PORT = os.getenv('MONG_PORT')
DATABASE = "tweetDumpingGround"

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId

uri = f"mongodb://{MONG_USER}:{MONG_PWD}@{MONG_HOST}:{MONG_PORT}/{DATABASE}"
client = MongoClient(uri)
db = client[DATABASE]
db

In [ ]:
from sys import maxsize as inf

LIMIT = 1 # inf
USER_ID = 'billwurtz'

## Helper functions

In [ ]:
def compress_object(origin_obj, attrs_to_save):
    new_obj = {'Not found':[]}

    for key in attrs_to_save:
        if key in user_detail:
            new_obj[key] = origin_obj[key]
        else:
            new_obj['Not found'].append(key)
    
    return new_obj

## Get use info

In [ ]:
user = api.get_user(USER_ID)
user_detail = user._json

In [ ]:
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user
attrs_to_save = [
    'id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description',
    'protected', 'verified', 'followers_count', 'friends_count', 'listed_count',
    'favourites_count', 'statuses_count', 'created_at', 'profile_banner_url', 
    'profile_image_url_https', 'default_profile', 'default_profile_image',
    'withheld_in_countries', 'withheld_scope'
]

my_user_obj = compress_object(user._json, attrs_to_save)

from datetime import datetime, timezone, timedelta
my_user_obj['Last updated'] = datetime.now(tz=timezone(timedelta(hours=+7)))
my_user_obj

Save to collection "profile"

In [ ]:
profile_col = db["profile"]
key = {'id_str': my_user_obj['id_str']}
profile_col.update(key, my_user_obj, upsert=True)

# Get tweets

In [ ]:
'''
import json

with open(os.path.join(PATH, f'{USER_ID}_tweet_dump.txt'), 'w+', encoding='utf8') as file:
    for tweet in tweepy.Cursor(api.user_timeline, id=USER_ID).items(LIMIT):
        # parsed = json.loads(tweet._json)
        print(json.dumps(tweet._json, indent=4, sort_keys=True))

        # print(tweet)
        # print(dir(tweet))
        file.write(tweet.text +'\n')
'''

In [ ]:
import json

for tweet in tweepy.Cursor(api.user_timeline, id=USER_ID).items(LIMIT):
    # parsed = json.loads(tweet._json)
    print(json.dumps(tweet._json, indent=4, sort_keys=True))
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
    attrs = [
        'id', 'text', 'attachments', 'author_id', 'location', 'url', 'description',
        'protected', 'verified', 'followers_count', 'friends_count', 'listed_count',
        'favourites_count', 'statuses_count', 'created_at', 'profile_banner_url', 
        'profile_image_url_https', 'default_profile', 'default_profile_image',
        'withheld_in_countries', 'withheld_scope'
    ]
    my_tweet_obj = compress_object(tweet._json, attrs)